In [3]:
import numpy as np

In [4]:
# For getting how long each group of customers would spend
np.random.uniform(30,150) # Generate one random number between 30 and 150 under uniform distribution

144.19198397661827

In [5]:
class Tables:
    def __init__(self, num, size):
        self.num=num # No. of the table
        self.size = size # Size of the table: for group of up to 2, 4 or 6.
        self.currentStatus = None # Is the table occupied or not.
        self.time_remaining = 0 
        
    def busy(self):
        if self.currentStatus != None:
            return True
        else:
            return False
    
    def startNext(self, newtask):
        self.currentStatus = newtask
        self.time_remaining = newtask.time_needs() # here maybe generate a random time from 10 - 60 mins using generat

In [41]:
def table_generator(number_tables_2, number_tables_4, number_tables_6):
    table_2_list = []
    table_4_list = []
    table_6_list = [] 
    for i in range(number_tables_2):
        table_2_list.append(Tables(i,2))
    
    for i in range(number_tables_4):
        table_4_list.append(Tables(i+number_tables_2,4))
    
    for i in range(number_tables_6):
        table_6_list.append(Tables(i+number_tables_4+number_tables_6,6))
    
    return (table_2_list, table_4_list, table_6_list)

In [43]:
table_2_list, table_4_list, table_6_list = table_generator(2,4,5)

In [44]:
table_2_list[0].num

0

In [45]:
table_4_list[0].num

2

In [46]:
class Queue:
    def __init__(self):
        self.groups=[]
    
    def queue_size(self):
        return len(self.item)
    
    def isEmpty(self):
        if len(self.item)>0:
            return True
        else:
            False
    
    def add_queue(self, group):
        if group.get_vip():
            self.groups.insert(4,group) # If current group is a VIP group, move it forward by four groups
        else:
            self.groups.insert(0, group)
        
    
    def del_queue(self): #delete last=delete first come group 
        self.groups.pop()

In [47]:
# this class can be regarded as Task in that case
import random

class Group:
    def __init__(self, time, size):
        self.timestamp = time # Time when group registered (entered into the queue)
        self.size = size #randomly define size from 1 - 6
    
    def get_stamp(self):
        return self.timestamp
    
    def get_size(self):
        return self.size
    
    def wait_time(self, current_time):
        return current_time - self.timestamp

In [49]:
def newGroup():
    num = np.random.normal(120,15,60)
    if num == 60:
        return True
    else:
        return False

In [73]:
int(abs((np.random.normal(120,45,30))))

TypeError: only length-1 arrays can be converted to Python scalars

In [ ]:
def simulation(Time_duration):
    customer_queue = Queue()
    waiting_time = []